In [23]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

from gpytorch.kernels import *
import gpytorch


train_X = torch.rand(10, 1)
train_X


tensor([[0.2927],
        [0.3899],
        [0.0430],
        [0.9168],
        [0.3758],
        [0.2978],
        [0.9761],
        [0.7227],
        [0.5727],
        [0.3702]])

In [24]:
Y = 1 - torch.norm(train_X - 0.5, dim=-1, keepdim=True)
Y = Y + 0.1 * torch.randn_like(Y)  # add some noise
Y



tensor([[0.8396],
        [0.8966],
        [0.4714],
        [0.6848],
        [0.8158],
        [0.7879],
        [0.3636],
        [0.7990],
        [0.9918],
        [0.8230]])

In [25]:
train_Y = standardize(Y)
train_Y

tensor([[ 0.4797],
        [ 0.7765],
        [-1.4355],
        [-0.3254],
        [ 0.3561],
        [ 0.2111],
        [-1.9963],
        [ 0.2686],
        [ 1.2714],
        [ 0.3937]])

In [26]:
# use a Matérn kernel with variance 0.5, lengthscale 0.5 and smoothness parameter 2.5
gp = SingleTaskGP(train_X, train_Y, covar_module=gpytorch.kernels.MaternKernel(nu=2.5, lengthscale_constraint=gpytorch.constraints.Interval(0.49, 0.51), outputscale_constraint=gpytorch.constraints.Interval(0.49, 0.51)))
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)



UCB = UpperConfidenceBound(gp, beta=0.1)



bounds = torch.stack([torch.zeros(1), torch.ones(1)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)
candidate  # tensor([0.4887, 0.5063])

c:\ProgramData\Anaconda3\lib\site-packages\botorch\models\gpytorch.py:113: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(


tensor([[0.5172]])

In [27]:
aa = torch.tensor([3,4])
aa

tensor([3, 4])